# Variables

In [1]:
ga_input = '/grid/home/nbourgeois/data/test_proteins/pah/pah.pep.fas' #sequence pep.fas Input
tissue = 'Liver' # Tissue type for CoCoPuts
filename = 'bai_1'

# Imports


In [2]:
import os
cwd = os.getcwd()

## Import the code for codon opt
import sys
sys.path.insert(1, '/grid/home/nbourgeois/codonOpt')
# from run_pygad import *
from general_functions import *
from dyn_prog import *
from run_pygad import *


In [3]:
(keys, seqs) = readFasta(ga_input)
if len(seqs) == 1:
    aa_seq=str(seqs[0])

In [4]:
wt_loc = '/grid/home/nbourgeois/data/codon_jason/wt_pah.fa'
a,seq = readFasta(wt_loc)
wt_seq = str(seq[0])

## Test Max

In [5]:
optimizer = Optimizer(aa_seq, tissues='Liver')
optimized_seq = optimizer.optimize()

## Test Ramp

In [6]:
optimizer = Optimizer(aa_seq, tissues='Liver', ramp=True)
optimized_seq = optimizer.optimize()

## Test OG Ramp

In [7]:
optimizer = Optimizer(aa_seq, tissues='Liver', og_ramp=True, og_seq=wt_seq)
optimized_seq = optimizer.optimize()

## Test Multi Weight

In [8]:
weights = pd.read_csv('test_data/weights.txt',sep='\t')
tissue_list = weights['Tissue'].values.tolist()
prefix_codon = 'AAG'

In [9]:
optimizer = Optimizer(aa_seq, tissues=tissue_list, prefix_codon=prefix_codon)
optimizer = Optimizer(aa_seq, tissues=tissue_list, ntissues='Liver', prefix_codon=prefix_codon)